In [ ]:

import tensorflow as tf

(x_train, y_train),(x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train, x_test = x_train / 255.0, x_test / 255.0

# 文件写入器
file_writer = tf.summary.create_file_writer("logs/2")
file_writer.set_as_default()

# 定义学习曲线，并且输出指标
def my_lr_schedule(epoch):
  learning_rate = 0.1
  learning_rate /= 2

  tf.summary.scalar('lr', data=learning_rate, step=epoch)
  return learning_rate

lr_callback = tf.keras.callbacks.LearningRateScheduler(my_lr_schedule)

model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=[])

# 调用学习率回调:callbacks=[lr_callback]
model.fit(
    x_train,
    y_train,
    epochs=3,
    validation_data=(x_test, y_test),
    callbacks=[lr_callback],
)

# 通过运行如下命令来查看结果：
# tensorboard --logdir logs/2
# 便可以查看到我们自定义的学习率指标的曲线的变化情况

In [ ]:
import tensorflow as tf
import numpy as np
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()
# 数据归一化
train_images = train_images / 255.0
test_images = test_images / 255.0

train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(64)

valid_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
valid_dataset = valid_dataset.batch(64)

model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

# 然后我们就可以定义我们的文件写入器，并且指定其日志写入目录：
file_writer = tf.summary.create_file_writer("logs/3")
file_writer.set_as_default()

'''
最后就到了我们的重头戏，我们可以编写自定义循环，
并且在自定义循环之中定义我们要输出的指标，以及输出的频率等：

在这里，我们在训练的过程之中使用 tf.summary.scalar 这个 API 进行指标的输出；
而且我们让其每 10个批次进行一次输出。
'''
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()
val_acc = tf.keras.metrics.SparseCategoricalAccuracy()
epochs = 3
for epoch in range(epochs):
    for batch_i, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
          outputs = model(x_batch_train, training=True)
          loss_value = loss_fn(y_batch_train, outputs)

        grads = tape.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

        if batch_i % 10 == 0:
            # 对训练数据进行Log：让其每 10个批次进行一次输出
            tf.summary.scalar('train_loss', data=float(loss_value), step=epoch*len(train_dataset)+batch_i)
            print("Loss at step %d: %.4f" % (epoch*len(train_dataset)+batch_i, float(loss_value)))
            

    for batch_i, (x_batch_train, y_batch_train) in enumerate(valid_dataset):
        # 对测试数据进行Log
        outputs = model(x_batch_train, training=False)
        val_acc.update_state(y_batch_train, outputs)
    tf.summary.scalar('valid_acc', data=float(val_acc.result()), step=epoch)
    val_acc.reset_states()
